In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import getpass
import psycopg2



In [3]:
data1 = pd.read_csv(r"real_estate_data_2023_01.csv")
data2 = pd.read_csv(r"real_estate_data_2023_02.csv")
data3 = pd.read_csv(r"real_estate_data_2023_03.csv")


In [162]:
data1.head()

,TransactionID,Date,PropertyID,PropertyName,PropertyType,LocationID,Address,City,State,ZipCode,SquareFeet,YearBuilt,SalePrice,AgentID,AgentName
0,2071,2023-01-01,308,Property_14,Land,128,4836 River Rd,Dallas,TX,44740,7441,2022,353392,66,Elizabeth Johnson
1,6853,2023-01-01,316,Property_338,Land,108,9216 Maple St,Los Angeles,CA,88329,6803,1973,306228,93,William Garcia
2,6031,2023-01-01,443,Property_42,Commercial,154,2765 Sunset Blvd,Houston,TX,64527,6597,1920,780815,89,James Davis
3,6024,2023-01-01,78,Property_276,Commercial,106,7994 Cedar St,Chicago,IL,77577,8679,2000,405708,26,Robert Davis
4,649,2023-01-01,85,Property_207,Commercial,85,7151 8th Ave,San Jose,CA,78698,4951,1906,353833,73,William Smith


In [163]:
data = pd.concat([data1, data2, data3])

In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2442 entries, 0 to 823
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TransactionID  2442 non-null   int64 
 1   Date           2442 non-null   object
 2   PropertyID     2442 non-null   int64 
 3   PropertyName   2442 non-null   object
 4   PropertyType   2442 non-null   object
 5   LocationID     2442 non-null   int64 
 6   Address        2442 non-null   object
 7   City           2442 non-null   object
 8   State          2442 non-null   object
 9   ZipCode        2442 non-null   int64 
 10  SquareFeet     2442 non-null   int64 
 11  YearBuilt      2442 non-null   int64 
 12  SalePrice      2442 non-null   int64 
 13  AgentID        2442 non-null   int64 
 14  AgentName      2442 non-null   object
dtypes: int64(8), object(7)
memory usage: 305.2+ KB


In [165]:
data.head()

,TransactionID,Date,PropertyID,PropertyName,PropertyType,LocationID,Address,City,State,ZipCode,SquareFeet,YearBuilt,SalePrice,AgentID,AgentName
0,2071,2023-01-01,308,Property_14,Land,128,4836 River Rd,Dallas,TX,44740,7441,2022,353392,66,Elizabeth Johnson
1,6853,2023-01-01,316,Property_338,Land,108,9216 Maple St,Los Angeles,CA,88329,6803,1973,306228,93,William Garcia
2,6031,2023-01-01,443,Property_42,Commercial,154,2765 Sunset Blvd,Houston,TX,64527,6597,1920,780815,89,James Davis
3,6024,2023-01-01,78,Property_276,Commercial,106,7994 Cedar St,Chicago,IL,77577,8679,2000,405708,26,Robert Davis
4,649,2023-01-01,85,Property_207,Commercial,85,7151 8th Ave,San Jose,CA,78698,4951,1906,353833,73,William Smith


In [166]:
data["Date"]= pd.to_datetime(data["Date"])

In [167]:
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
data['quarter'] = data['Date'].dt.quarter

In [168]:
data.columns

Index(['TransactionID', 'Date', 'PropertyID', 'PropertyName', 'PropertyType',
       'LocationID', 'Address', 'City', 'State', 'ZipCode', 'SquareFeet',
       'YearBuilt', 'SalePrice', 'AgentID', 'AgentName', 'year', 'month',
       'quarter'],
      dtype='object')

# Detecting the entities from the columns 

`location_dim` = LocationID, Address, City, State, Zipcode

`Property_dim` = PropertyID, 'PropertyName', 'PropertyType', YearBuilt

`agent_dim` = 'AgentID', 'AgentName'

`date_dim` = 'Date', 'year', 'month', 'quarter'

`fact_transactions` = 'TransactionID', 'SalePrice', 'SquareFeet', 'LocationID, 'AgentID', PropertyID, Date'




In [169]:
property = data[[ 'PropertyID', 'PropertyName', 'PropertyType', 'YearBuilt']]
property

,PropertyID,PropertyName,PropertyType,YearBuilt
0,308,Property_14,Land,2022
1,316,Property_338,Land,1973
2,443,Property_42,Commercial,1920
3,78,Property_276,Commercial,2000
4,85,Property_207,Commercial,1906
...,...,...,...,...
819,481,Property_89,Land,1992
820,51,Property_98,Land,1904
821,410,Property_71,Land,2010
822,474,Property_282,Residential,1908


In [170]:
location = data[[ 'LocationID', 'Address', 'City', 'State', 'ZipCode']]
location

,LocationID,Address,City,State,ZipCode
0,128,4836 River Rd,Dallas,TX,44740
1,108,9216 Maple St,Los Angeles,CA,88329
2,154,2765 Sunset Blvd,Houston,TX,64527
3,106,7994 Cedar St,Chicago,IL,77577
4,85,7151 8th Ave,San Jose,CA,78698
...,...,...,...,...,...
819,5,9406 Pine St,Dallas,TX,36347
820,196,8565 2nd St,Phoenix,AZ,58935
821,8,8523 River Rd,New York,NY,26034
822,47,1673 8th Ave,San Antonio,TX,96582


In [171]:
agent = data[[ 'AgentID', 'AgentName']]

In [172]:
date = data[['Date', 'year', 'month', 'quarter']]
date

,Date,year,month,quarter
0,2023-01-01,2023,1,1
1,2023-01-01,2023,1,1
2,2023-01-01,2023,1,1
3,2023-01-01,2023,1,1
4,2023-01-01,2023,1,1
...,...,...,...,...
819,2023-03-31,2023,3,1
820,2023-03-31,2023,3,1
821,2023-03-31,2023,3,1
822,2023-03-31,2023,3,1


In [173]:
transactions = data[['TransactionID', 'SalePrice', 'SquareFeet', 'LocationID', 'AgentID', 'PropertyID', 'Date']]

In [174]:
property = property.drop_duplicates('PropertyID')
location = location.drop_duplicates('LocationID')
agent = agent.drop_duplicates('AgentID')
date = date.drop_duplicates('Date')
transactions = transactions.drop_duplicates('TransactionID')

In [179]:
date.rename(columns={'Date': 'transaction_date'}, inplace=True)
date.head()

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,transaction_date,year,month,quarter
0,2023-01-01,2023,1,1
30,2023-01-02,2023,1,1
56,2023-01-03,2023,1,1
81,2023-01-04,2023,1,1
106,2023-01-05,2023,1,1


In [177]:
transactions

,TransactionID,SalePrice,SquareFeet,LocationID,AgentID,PropertyID,Date
0,2071,353392,7441,128,66,308,2023-01-01
1,6853,306228,6803,108,93,316,2023-01-01
2,6031,780815,6597,154,89,443,2023-01-01
3,6024,405708,8679,106,26,78,2023-01-01
4,649,353833,4951,85,73,85,2023-01-01
...,...,...,...,...,...,...,...
819,3868,585501,6987,5,40,481,2023-03-31
820,3882,400403,6263,196,86,51,2023-03-31
821,8555,653512,8525,8,88,410,2023-03-31
822,6076,759365,1261,47,20,474,2023-03-31


In [178]:
transactions.rename(columns={'Date': 'transaction_date'}, inplace=True)
transactions.head()

,TransactionID,SalePrice,SquareFeet,LocationID,AgentID,PropertyID,transaction_date
0,2071,353392,7441,128,66,308,2023-01-01
1,6853,306228,6803,108,93,316,2023-01-01
2,6031,780815,6597,154,89,443,2023-01-01
3,6024,405708,8679,106,26,78,2023-01-01
4,649,353833,4951,85,73,85,2023-01-01


In [189]:
transactions['transaction_date'] = transactions['transaction_date'].dt.date
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2442 entries, 0 to 823
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TransactionID     2442 non-null   int64 
 1   SalePrice         2442 non-null   int64 
 2   SquareFeet        2442 non-null   int64 
 3   LocationID        2442 non-null   int64 
 4   AgentID           2442 non-null   int64 
 5   PropertyID        2442 non-null   int64 
 6   transaction_date  2442 non-null   object
dtypes: int64(6), object(1)
memory usage: 152.6+ KB


In [212]:
password= getpass.getpass()

········


In [215]:
try:
    conn = psycopg2.connect(
    host="localhost",
    database="cricket",
    port = 5433,
    user="postgres",
    password= password
    )
    print('connection successful')
except Exception as e:
     print(f' Error: {e}')

connection successful


In [216]:
cursor = conn.cursor()  #

create_table_query = """

CREATE TABLE IF NOT EXISTS date (
    transaction_date DATE PRIMARY KEY,  -- Use DATE instead of DATETIME since it's just a reference
    year INT,
    month INT, 
    quarter INT
);

CREATE TABLE IF NOT EXISTS property (
    PropertyID INT PRIMARY KEY,
    PropertyName VARCHAR(255),
    PropertyType VARCHAR(255),
    YearBuilt INT
);

CREATE TABLE IF NOT EXISTS agent (
    AgentID INT PRIMARY KEY,
    AgentName VARCHAR(255)
);

CREATE TABLE IF NOT EXISTS location (
    LocationID INT PRIMARY KEY,
    Address VARCHAR(255),
    City VARCHAR(255),
    State VARCHAR(255),
    ZipCode INT
);

CREATE TABLE IF NOT EXISTS transactions (
    TransactionID INT PRIMARY KEY,        
    SalePrice INT,          
    SquareFeet INT,     
    LocationID INT,  -- Foreign key referring to 'location' table
    AgentID INT,     -- Foreign key referring to 'agent' table
    PropertyID INT,  -- Foreign key referring to 'property' table
    transaction_date DATE,
    
    FOREIGN KEY (LocationID) REFERENCES location(LocationID) ON DELETE CASCADE,
    FOREIGN KEY (AgentID) REFERENCES agent(AgentID) ON DELETE CASCADE,
    FOREIGN KEY (PropertyID) REFERENCES property(PropertyID) ON DELETE CASCADE,
    FOREIGN KEY (transaction_date) REFERENCES date(transaction_date) ON DELETE CASCADE
);


"""

cursor.execute(create_table_query)
conn.commit()

print("Table 'table' created successfully.")



Table 'table' created successfully.


In [207]:
for _, row in property.iterrows():
    cursor.execute(
        """
        INSERT INTO property (PropertyID, PropertyName, PropertyType, YearBuilt) 
        VALUES (%s, %s, %s, %s)
        """, 
        (row['PropertyID'], row['PropertyName'], row['PropertyType'], row['YearBuilt'])
    )

for _, row in date.iterrows():
    cursor.execute(
        """
        INSERT INTO date (transaction_date, year, month, quarter) 
        VALUES (%s, %s, %s, %s)
        """, 
        (row['transaction_date'], row['year'], row['month'], row['quarter'])
    )

for _, row in agent.iterrows():
    cursor.execute(
        """
        INSERT INTO agent (AgentID, AgentName) 
        VALUES (%s, %s)
        """, 
        (row['AgentID'], row['AgentName'])
    )

for _, row in location.iterrows():
    cursor.execute(
        """
        INSERT INTO location (LocationID, Address, City, State, ZipCode) 
        VALUES (%s, %s, %s, %s, %s)
        """, 
        (row['LocationID'], row['Address'], row['City'], row['State'], row['ZipCode'])
    )
    

conn.commit()

print("CSV file successfully loaded into PostgreSQL.")

# Close connections
cursor.close()
conn.close()


CSV file successfully loaded into PostgreSQL.


In [217]:
for _, row in transactions.iterrows():
    cursor.execute(
        """
        INSERT INTO transactions (TransactionID, SalePrice, SquareFeet, LocationID, AgentID, PropertyID, transaction_date) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (row['TransactionID'], row['SalePrice'], row['SquareFeet'], row['LocationID'], row['AgentID'], row['PropertyID'], row['transaction_date'])   
    )

conn.commit()

print("CSV file successfully loaded into PostgreSQL.")

# Close connections
cursor.close()
conn.close()


CSV file successfully loaded into PostgreSQL.


In [4]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TransactionID  859 non-null    int64 
 1   Date           859 non-null    object
 2   PropertyID     859 non-null    int64 
 3   PropertyName   859 non-null    object
 4   PropertyType   859 non-null    object
 5   LocationID     859 non-null    int64 
 6   Address        859 non-null    object
 7   City           859 non-null    object
 8   State          859 non-null    object
 9   ZipCode        859 non-null    int64 
 10  SquareFeet     859 non-null    int64 
 11  YearBuilt      859 non-null    int64 
 12  SalePrice      859 non-null    int64 
 13  AgentID        859 non-null    int64 
 14  AgentName      859 non-null    object
dtypes: int64(8), object(7)
memory usage: 100.8+ KB
